## Old script

In [ ]:
import json
from oauth2client.service_account import ServiceAccountCredentials
import base64
from datetime import datetime
import boto3
from botocore.exceptions import ClientError
import importlib
import psycopg2
import psycopg2.extras as extras
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta, date
import time
from requests import get
from pathlib import Path
import gspread
import sys
import gzip
import http.client
import sqlalchemy
import os
import math
import snowflake.connector

# get secret function:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return base64.b64decode(get_secret_value_response['SecretBinary'])


def get_from_gsheet_edited(workbook, sheet):
    scope = [
        "https://spreadsheets.google.com/feeds",
        'https://www.googleapis.com/auth/spreadsheets',
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"]

    json_path2 = "/tmp//maxab_sheets_key.json"
    print(json_path2)
    maxab_sheets_key = get_secret("prod/maxab-sheets")
    f = open(json_path2, "w")
    f.write(maxab_sheets_key)
    f.close()
    os.environ["MAXAB_SHEETS_KEY_PATH"] = json_path2

    creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["MAXAB_SHEETS_KEY_PATH"], scope)
    client = gspread.authorize(creds)

    wks = client.open(workbook).worksheet(sheet)
    sheet = pd.DataFrame(wks.get_all_values())
    sheet.columns = sheet.iloc[0]
    sheet = sheet[1:]
    sheet = sheet[sheet.columns[list(map(lambda x: x != '', sheet.columns))]]
    return sheet


def send_custom_events(custom_events):
    try:
        base_url = 'https://go.urbanairship.com'
        base_headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer ' + os.environ["AIRSHIP_ACCESS_TOKEN"],
            'X-UA-Appkey': os.environ["AIRSHIP_APP_KEY"],
            'Accept': 'application/vnd.urbanairship+json; version=3'
        }
        s_response = requests.post(
            base_url + '/api/custom-events',
            data=json.dumps(custom_events),
            headers=base_headers
        )
        s_response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)


def send_custom_events_lite(custom_events):
    try:
        base_url = 'https://go.urbanairship.com'
        base_headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer '+os.environ["AIRSHIP_ACCESS_TOKEN_LITE"],
            'X-UA-Appkey': os.environ["AIRSHIP_APP_KEY_LITE"],
            'Accept': 'application/vnd.urbanairship+json; version=3'
        }
        s_response = requests.post(
            base_url + '/api/custom-events',
            data=json.dumps(custom_events),
            headers=base_headers
        )
        s_response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)


def tag_airship_users(user_tags):
    try:
        base_url = 'https://go.urbanairship.com'
        base_headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer ' + os.environ["AIRSHIP_ACCESS_TOKEN"],
            'X-UA-Appkey': os.environ["AIRSHIP_APP_KEY"],
            'Accept': 'application/vnd.urbanairship+json; version=3'
        }
        s_response = requests.post(
            base_url + '/api/named_users/tags',
            data=json.dumps(user_tags),
            headers=base_headers
        )
        s_response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)


def build_custom_event(event_properties, main_system_id, event_name):
    named_user = "EG_retailers_" + str(main_system_id)
    # named_user =  "EG_retailers_{}".format(main_system_id)
    retailer_event = {}
    retailer_event['occurred'] = str(datetime.now().strftime("%Y-%m-%dT%H:%M:%S"))
    retailer_event['user'] = {"named_user_id": str(named_user)}

    retailer_event['body'] = {"name": event_name, "properties": event_properties}

    return retailer_event


def lambda_handler(event, context):
    # dwh connection:
    dwh_reader_secret_new = json.loads(get_secret("prod/db/datawarehouse/metabase"))

    dwh_writer_secret_new = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))
    host=dwh_reader_secret_new["host"]
    database=dwh_reader_secret_new["dbname"]
    user=dwh_reader_secret_new["username"]
    password=dwh_reader_secret_new["password"]

    conn = psycopg2.connect(host=host, database=database, user=user, password=password)
    print("Successfully connected to DB")

    # snowflake connection:
    snowflake_secret = json.loads(get_secret("prod/snowflake/fintech-credentials"))
    sf_account = snowflake_secret["snowflake_account"]
    sf_warehouse = snowflake_secret["snowflake_fintech_warehouse"]
    sf_database = snowflake_secret["snowflake_database"]
    sf_username = snowflake_secret["snowflake_fintech_username"]
    sf_password = snowflake_secret["snowflake_fintech_password"]

    snowflake_con = snowflake.connector.connect(
        user=sf_username,
        account=sf_account,
        password=sf_password,
        database=sf_database,
        warehouse=sf_warehouse
    )

    # airship connection:
    airship_secret = json.loads(get_secret("prod/airship/accessToken"))

    os.environ["AIRSHIP_APP_KEY"] = airship_secret['AIRSHIP_APP_KEY']
    os.environ["AIRSHIP_ACCESS_TOKEN"] = airship_secret['AIRSHIP_ACCESS_TOKEN']

    # airship connection lite version:

    airship_secret = json.loads(get_secret("prod/airship/accessToken/lite"))

    os.environ["AIRSHIP_APP_KEY_LITE"] = airship_secret['AIRSHIP_APP_KEY_LITE']
    os.environ["AIRSHIP_ACCESS_TOKEN_LITE"] = airship_secret['AIRSHIP_ACCESS_TOKEN_LITE']

    # get drivers sheet data:
    collection_agents_personal_info = get_from_gsheet_edited("collection_agents_personal_info", "Sheet1")

    # getting records:
    for record in event['Records']:
        data = record['dynamodb']['NewImage']
        print(data)
        print(event)

        for key in list(data.keys()):
            if key in [
                'retailer_id',
                'collection_order_status_id',
                'collection_amount',
                'collected_amount',
                'created_at',
                'updated_at',
                'initial_date',
                'cashin_mode_id',
                'id'
            ]:
                data[key] = list(data[key].values())[0]
            else:
                del data[key]

        print(data)
        # get collection order id:

        collection_order_status_id = data['collection_order_status_id']
        collection_order_id = data['id']
        main_system_id = data['retailer_id']

        # get agent information:
        agent_mobile_query = '''select
            national_id,
            mobile,
            name,
            drivers.id,
            split_part(name,' ',1) first_name,
            split_part(name,' ',2) last_name,
            right(national_id,7) as last_7_national_id
        from public.drivers
            join public.run_sheets on run_sheets.driver_id = drivers.id 
            join public.collection_orders on collection_orders.run_sheet_id = run_sheets.id

        where collection_orders.id={} '''.format(collection_order_id)

        try:
            agent_mobile_query_result = pd.read_sql(agent_mobile_query, conn)

            # agent_first_name = agent_mobile_query_result.loc[0,'first_name']
            # agent_last_name= agent_mobile_query_result.loc[0,'last_name']

            agent_last_7_digits_nid = agent_mobile_query_result.loc[0, 'last_7_national_id']
            agent_nid = agent_mobile_query_result.loc[0, 'national_id']
            agent_full_name_ar = collection_agents_personal_info.loc[collection_agents_personal_info.national_id==agent_nid, 'name_ar'].values.tolist()[0]
            agent_first_name_ar = agent_full_name_ar.split(" ")[0]
            agent_second_name_ar = agent_full_name_ar.split(" ")[1]
            agent_third_name_ar = agent_full_name_ar.split(" ")[2]
            agent_mobile = agent_mobile_query_result.loc[0, 'mobile']
            agent_id = str(agent_mobile_query_result.loc[0, 'id'])

            # add variables to the trigger:
            data['agent_id'] = agent_id
            data['agent_mobile'] = agent_mobile
            data['agent_last_7_digits_nid'] = agent_last_7_digits_nid
            data['agent_first_name_ar'] = agent_first_name_ar
            data['agent_second_name_ar'] = agent_second_name_ar
            data['agent_third_name_ar'] = agent_third_name_ar

            print(data)
            # create the trigger:
            custom_event = build_custom_event(data, main_system_id, "db_trigger_collection_order")

            # send the trigger to airship:
            send_custom_events(custom_event)

            # send the trigger to airship lite version:
            send_custom_events_lite(custom_event)
        except Exception as e:
            print(e)

    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }

## New script

In [7]:
import json
import base64
import boto3
from botocore.exceptions import ClientError
import http.client
import psycopg2
import sqlalchemy
import os
import pandas as pd
import numpy as np
import math
import time
import re 
# import snowflake.connector
from datetime import datetime

dynamodb = boto3.resource('dynamodb')
dynamodb_table = dynamodb.Table('db_trigger')

# sending sms:
def send_message(messagesObject, tag=None):
    auth = "App b9c03ee856baa2b7ac96b4b0ecb8aa36-ce2dbf98-1fd2-4fff-9b8b-0d5b8187c834"
    conn = http.client.HTTPSConnection("qg3p63.api.infobip.com")
    if tag is None:
        payload = json.dumps({"messages": messagesObject})
    else:
        payload = json.dumps({"bulkId": tag, "messages": messagesObject})

    headers = {
        'Authorization': "App b9c03ee856baa2b7ac96b4b0ecb8aa36-ce2dbf98-1fd2-4fff-9b8b-0d5b8187c834",
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    conn.request("POST", "/sms/2/text/advanced", payload, headers)
    res = conn.getresponse()
    data = res.read()
    final = eval(data)
    print(data)
    print(final['bulkId'])

    return final['bulkId']
    
# get secret function:
def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            raise e
    else:
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return base64.b64decode(get_secret_value_response['SecretBinary']) 

# get message result:
# get message result:
def get_sms_results(bulkId=None, messageId=None, after_timestamp=None):
    # Authentication
    conn = http.client.HTTPSConnection("qg3p63.api.infobip.com")
    payload = ''
    headers = {
        'Authorization': 'App b9c03ee856baa2b7ac96b4b0ecb8aa36-ce2dbf98-1fd2-4fff-9b8b-0d5b8187c834',
        'Accept': 'application/json'
    }

    # Get reports based on messageId:
    if messageId:
        # Connect with the API and get the response based on bulkId:
        conn.request("GET", "/sms/1/logs?messageId={}".format(messageId), payload, headers)
        res = conn.getresponse()
        data = res.read()
        d = json.loads(data)
        df = pd.DataFrame(d['results'])
        new_col_names = [
            'bulk_id',
            'message_id',
            'to',
            'from',
            'text',
            'sent_at',
            'done_at',
            'sms_count',
            'price',
            'status',
            'error',
            'application_id'
        ]

        
        try:
            # drop mccMnc:
            df.drop(columns=['mccMnc'], inplace=True)

            # drop the country code (2):
            df.to = df.to.str[1:]
            
            # change columns names:
            df.columns = new_col_names
            
            # data wrangling:
            # error column cleaning
            df['error'] = df['error'].apply(lambda x: x['name'])

            # status:
            df['status'] = df['status'].apply(lambda x: x['groupName'])

            # price:
            df['price'] = df['price'].apply(lambda x: x['pricePerMessage'])

            # dates cleaning:
            df['sentAt'] = pd.to_datetime(df['sentAt'], format='%Y-%m-%dT%H:%M:%S').apply(lambda x: x.replace(tzinfo=None))
            df['doneAt'] = pd.to_datetime(df['doneAt'], format='%Y-%m-%dT%H:%M:%S').apply(lambda x: x.replace(tzinfo=None))

            
        except Exception as e:
            print('error e'.format(e))

    # Get reports base on bulkId:
    elif bulkId:
        # Connect with the API and get the response based on bulkId:
        conn.request("GET", "/sms/1/logs?bulkId={}".format(bulkId), payload, headers)
        res = conn.getresponse()
        data = res.read()
        d = json.loads(data)
        df = pd.DataFrame(d['results'])
        new_col_names = [
            'bulk_id',
            'message_id',
            'to',
            'from',
            'text',
            'sent_at',
            'done_at',
            'sms_count',
            'price',
            'status',
            'error',
            'application_id'
        ]

        try:
            # drop mccMnc:
            df.drop(columns=['mccMnc'], inplace=True)

            # drop the country code (2):
            df.to = df.to.str[1:]
            
            # change columns names:
            df.columns = new_col_names
            
            # data wrangling:
            # error column cleaning:
            df['error'] = df['error'].apply(lambda x: x['name'])

            # status:
            df['status'] = df['status'].apply(lambda x: x['groupName'])

            # price:
            df['price'] = df['price'].apply(lambda x: x['pricePerMessage'])

            # dates cleaning:
            df['sentAt'] = pd.to_datetime(df['sentAt'], format='%Y-%m-%dT%H:%M:%S.%f%z').apply(lambda x: x.replace(tzinfo=None))
            df['doneAt'] = pd.to_datetime(df['doneAt'], format='%Y-%m-%dT%H:%M:%S.%f%z').apply(lambda x: x.replace(tzinfo=None))

            
        except Exception as e:
            print('error {}'.format(e))

    else:
        return print("error in inputs")

    if after_timestamp:
        after_timestamp = datetime.datetime.strptime(after_timestamp, '%Y-%m-%d %H:%M:%S.%f')
        df_subset = df.loc[df.sent_at >= after_timestamp]

    else:
        df_subset = df

    return df_subset

In [12]:
def lambda_handler(event):
    
    # connect to the dwh db:
    dwh_reader_secret_new = json.loads(get_secret("prod/db/datawarehouse/metabase"))
    dwh_writer_secret_new = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))

    # get secrets: -dwh reader
    host_r=dwh_reader_secret_new["host"]
    database_r=dwh_reader_secret_new["dbname"]
    user_r=dwh_reader_secret_new["username"]
    password_r=dwh_reader_secret_new["password"]
    
    # get secrets: -dwh writer
    host_w=dwh_writer_secret_new["host"]
    database_w=dwh_writer_secret_new["dbname"]
    user_w=dwh_writer_secret_new["username"]
    password_w=dwh_writer_secret_new["password"]

    conn_r = psycopg2.connect(host=host_r, database=database_r, user=user_r, password=password_r)
    print("Successfully connected to reader DB")
    
    conn_w = psycopg2.connect(host=host_w, database=database_w, user=user_w, password=password_w)
    print("Successfully connected to writer DB")
    
        
#     # check if there is a need for the message:
#     try:
#         linked_at_id, retailer_id, linked_device_id, device_type, device_manufacturer, mobile  = get_inputs_from_event(event, conn_r)
#         # get_inputs_from_event(event,conn_r)
#     except Exception as e:
#         print("error in getting full_result:{}".format(e))
#         check_validity = get_inputs_from_event(event, conn_r)
    
    valid_records_list=[]
    for record in event['Records']:
        data = record['dynamodb']['NewImage']
        print(data) 
        print(event)
        
        #extract relevant data 
        collection_order_id = list(data['id'].values())[0]
        updated_at = list(data['updated_at'].values())[0]
        collection_order_updated_at = datetime.strptime(updated_at, "%Y-%m-%dT%H:%M:%SZ")
        collection_order_updated_at = collection_order_updated_at.date()
        retailer_id =  list(data['retailer_id'].values())[0]
        run_sheet_id = list(data['run_sheet_id'].values())[0]
    
        if(collection_order_updated_at != datetime.today().date()):

            print('collection order wasnt updated today')
            continue 

        #get the retailer mobile number 

        get_mobile_sql = '''
        select distinct r.retailer_id , r.mobile 
        from public.retailers r
        where r.id = {}
        '''.format(retailer_id)

        get_mobile_sql_result = pd.read_sql(get_mobile_sql, conn_r)
        main_system_id = get_mobile_sql_result['retailer_id'][0]
        mobile = get_mobile_sql_result['mobile'][0]
        
#     print(linked_at_id, linked_at, retailer_id, linked_device_id,mobile,main_system_id,device_type,device_manufacturer )
  

#     #check if there is a need to send the sms 
#     #check comms rules:
#     check_comm_counter_sql = '''
#      select distinct
#         receiver_id,
#         ref_entity,
#         ref_id,
#         receiver_phone,
#         date(cast(sending_time as date)) as sending_date,
#         count(case when message_status in ('PENDING','DELIVERED') then 1 else null end) over(partition by receiver_id, date_trunc('day',cast(sending_time as date))) as successful_sms_per_day,
#         count(*) over(partition by receiver_id, date_trunc('day',cast(sending_time as date))) as sms_per_day,
#         count(case when message_status in ('PENDING','DELIVERED') then 1 else null end) over(partition by receiver_id,ref_entity, ref_id) as successful_sms_per_entity_per_ref_id,
#         count(*) over(partition by receiver_id,ref_entity, ref_id) as sms_per_entity_per_ref_id
#     from fintech.fintech_communication_logs
#     where ref_entity = 'device_linked_at_id'
#         and communication_reason = 'new_device_notification_test'
#         and date(sending_time) = date(now()) 
#         and ref_id = {}'''.format(linked_at_id)

#     try:
#         check_comm_counter_sql_result = pd.read_sql(check_comm_counter_sql, conn_r)
#         successful_sms_per_entity_per_ref_id = check_comm_counter_sql_result['successful_sms_per_entity_per_ref_id'][0]
#     except Exception as e:
#         print("error in comm counter result:{}".format(e))
#         successful_sms_per_entity_per_ref_id = 0

#     if successful_sms_per_entity_per_ref_id >= 1:
#         print("message has already been sent")
#         return "max_comm_validation"

    
#     # Send SMS:
    
#     # generate message object:
# #     receiver_mobile = "2"+mobile
#     receiver_mobile = "201112255939"

#     if device_type == 'MOBILE':
        
#         sms_template = 'عميل مدفوعات، تم اضافة جهاز جديد {0} لحسابك. في حالة عدم الموافقة يرجى التواصل معنا على 15425 في اسرع وقت'
#         y = [{
#             'from':'MaxAB',
#             'destinations': [{"to": "{}".format(receiver_mobile)}], 'text': sms_template.format(device_manufacturer)}]
    
#     else: 
        
#         sms_template = 'عميل مدفوعات، تم اضافة ماكينة جديدة لحسابك. في حالة عدم الموافقة يرجى التواصل معنا على 15425 في اسرع وقت'
#         y = [{
#             'from':'MaxAB',
#             'destinations': [{"to": "{}".format(receiver_mobile)}], 'text': sms_template}]

#     message_tag = "new_device_notification_{}".format(linked_at_id)

#     send_message(y, tag=message_tag)
#     print('message_sent_to_{}'.format(linked_at_id))
#     time.sleep(20)

#     # get message result:
#     message_result = get_sms_results(bulkId=message_tag)
#     print(message_result)

#     try:
#         message_id = message_result[message_result.bulk_id == message_tag]['message_id'][0]
#     except:
#         message_id = 0
#     try:
#         sending_time = message_result[message_result.bulk_id == message_tag]['sent_at'][0]
#     except:
#         sending_time = datetime.now()
#     try:
#         message_status = message_result[message_result.bulk_id == message_tag]['status'][0]
#     except:
#         message_status = 'Unkown'


#     # Write SMS Logs to DB 
#     # db connection:
    
#     engine_w = sqlalchemy.create_engine(f"postgresql+psycopg2://{user_w}:{password_w}@{host_w}/{database_w}")
#     print(bool(engine_w))
    
#     with engine_w.connect() as conn:
#         print("start writing into fintech sms logs table")
#         message_result.to_sql(name='fintech_sms_logs', schema='fintech', con=conn, if_exists='append', chunksize=1000, method='multi')
#         print("written into fintech sms logs table successfully")

#     # push comm log to db
#     comm_df = pd.DataFrame(columns=[
#         'receiver_type',
#         'receiver_id',
#         'receiver_phone',
#         'communication_reason',
#         'ref_id',
#         'ref_entity',
#         'message_id',
#         'message_status',
#         'sending_time'])

#     comm_df.loc[len(comm_df)] = [
#         'retailer',
#         main_system_id,
#         mobile,
#         'new_device_notification_test',
#         linked_at_id,
#         'device_linked_at_id',
#         message_id,
#         message_status,
#         sending_time]

#     # update the table on db:
#     with engine_w.connect() as conn:
#         print("start writing into fintech communication logs table")
#         comm_df.to_sql(
#             name='fintech_communication_logs',
#             schema='fintech',
#             con=conn,
#             if_exists='append',
#             chunksize=1000,
#             method='multi')
#         print("written into fintech communication logs table successfully")



#     # close the connection:
#     conn_w.close()
#     conn_r.close()

#     return {
#         'statusCode': 200,
#         'body': json.dumps('Hello from Lambda!')
#     }

In [13]:
lambda_handler(
{
   "Records":[
      {
         "eventID":"20af2fd73916b0c9c7af91236a92538c",
         "eventName":"MODIFY",
         "eventVersion":"1.1",
         "eventSource":"aws:dynamodb",
         "awsRegion":"us-east-1",
         "dynamodb":{
            "ApproximateCreationDateTime":1716982811.0,
            "Keys":{
               "id":{
                  "N":"118562018"
               }
            },
            "NewImage":{
               "date":{
                  "S":"2024-05-29T00:00:00Z"
               },
               "is_partial":{
                  "NULL":True
               },
               "collection_order_status_id":{
                  "N":"3"
               },
               "cashin_mode_id":{
                  "N":"2"
               },
               "created_at":{
                  "S":"2024-05-28T01:55:05Z"
               },
               "run_sheet_id":{
                  "N":"1402353"
               },
               "locus_rank":{
                  "NULL":True
               },
               "collection_order_ttl":{
                  "N":"1716947705"
               },
               "updated_at":{
                  "S":"2024-05-29T14:40:10Z"
               },
               "confirmation_counter":{
                  "N":"0"
               },
               "collection_amount":{
                  "N":"1000"
               },
               "initial_date":{
                  "S":"2024-05-29T00:00:00Z"
               },
               "minimum_collection_amount":{
                  "NULL":True
               },
               "id":{
                  "N":"118562018"
               },
               "collected_amount":{
                  "N":"0"
               },
               "type_id":{
                  "N":"2"
               },
               "num_of_delays":{
                  "N":"0"
               },
               "collections_order_sub_status_id":{
                  "N":"1"
               },
               "retailer_id":{
                  "N":"287674"
               },
               "submitted_by":{
                  "S":"RETAILER"
               },
               "maximum_collection_amount":{
                  "NULL":True
               },
               "vehicle_type_id":{
                  "N":"34"
               },
               "issued_collection_amount":{
                  "N":"1000"
               },
               "maxab_collection_order_id":{
                  "N":"2997886"
               },
               "district_id":{
                  "N":"565"
               },
               "amount_to_be_collected":{
                  "NULL":True
               },
               "category_group_id":{
                  "N":"1"
               },
               "tag_english":{
                  "S":"Cash-in Request"
               },
               "warehouse_id":{
                  "N":"1"
               }
            },
            "OldImage":{
               "date":{
                  "S":"2024-05-29T00:00:00Z"
               },
               "type_id":{
                  "N":"2"
               },
               "collection_order_status_id":{
                  "N":"1"
               },
               "num_of_delays":{
                  "N":"0"
               },
               "cashin_mode_id":{
                  "N":"2"
               },
               "collections_order_sub_status_id":{
                  "N":"3"
               },
               "created_at":{
                  "S":"2024-05-28T01:55:05Z"
               },
               "retailer_id":{
                  "N":"287674"
               },
               "submitted_by":{
                  "S":"RETAILER"
               },
               "collection_order_ttl":{
                  "N":"1716947705"
               },
               "vehicle_type_id":{
                  "N":"34"
               },
               "updated_at":{
                  "S":"2024-05-28T01:55:05Z"
               },
               "confirmation_counter":{
                  "N":"0"
               },
               "collection_amount":{
                  "N":"1000"
               },
               "initial_date":{
                  "S":"2024-05-29T00:00:00Z"
               },
               "issued_collection_amount":{
                  "N":"1000"
               },
               "maxab_collection_order_id":{
                  "N":"2997886"
               },
               "district_id":{
                  "N":"565"
               },
               "id":{
                  "N":"118562018"
               },
               "category_group_id":{
                  "N":"1"
               },
               "tag_english":{
                  "S":"Cash-in Request"
               },
               "collected_amount":{
                  "N":"0"
               },
               "warehouse_id":{
                  "N":"1"
               }
            },
            "SequenceNumber":"571610300000000091106732594",
            "SizeBytes":1098,
            "StreamViewType":"NEW_AND_OLD_IMAGES"
         },
         "eventSourceARN":"arn:aws:dynamodb:us-east-1:876425898567:table/collection_orders/stream/2024-05-29T11:14:01.243"
      }
   ]
}
)

Successfully connected to reader DB
Successfully connected to writer DB
{'date': {'S': '2024-05-29T00:00:00Z'}, 'is_partial': {'NULL': True}, 'collection_order_status_id': {'N': '3'}, 'cashin_mode_id': {'N': '2'}, 'created_at': {'S': '2024-05-28T01:55:05Z'}, 'run_sheet_id': {'N': '1402353'}, 'locus_rank': {'NULL': True}, 'collection_order_ttl': {'N': '1716947705'}, 'updated_at': {'S': '2024-05-29T14:40:10Z'}, 'confirmation_counter': {'N': '0'}, 'collection_amount': {'N': '1000'}, 'initial_date': {'S': '2024-05-29T00:00:00Z'}, 'minimum_collection_amount': {'NULL': True}, 'id': {'N': '118562018'}, 'collected_amount': {'N': '0'}, 'type_id': {'N': '2'}, 'num_of_delays': {'N': '0'}, 'collections_order_sub_status_id': {'N': '1'}, 'retailer_id': {'N': '287674'}, 'submitted_by': {'S': 'RETAILER'}, 'maximum_collection_amount': {'NULL': True}, 'vehicle_type_id': {'N': '34'}, 'issued_collection_amount': {'N': '1000'}, 'maxab_collection_order_id': {'N': '2997886'}, 'district_id': {'N': '565'}, 'am

'collection order wasnt updated today'